In [26]:
import pandas as pd
import snowflake.connector

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_PROD',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [27]:
#read the data from csv file and save in mldf
mldf = pd.read_csv('modeleval2.csv')

In [30]:
#Get data from snowflake
sql = f'''
  SELECT
    status,
    created_at,
    USER_EMAIL,
    _ID,
    REPLACE(a.key, '__DOT__', '.') AS PVID,
    b.key AS ATTRIBUTE_NAME,
    c.key AS Audit_Key,
    c.value AS AuditResult
  FROM
    SCALE_CRAWLER.PUBLIC.SPOTTERAUDITS,
    LATERAL FLATTEN (INPUT => result) a,
    LATERAL FLATTEN (INPUT => a.value, OUTER => TRUE) b,
    LATERAL FLATTEN (INPUT => b.value, OUTER => TRUE) c,
    LATERAL FLATTEN (INPUT => c.value, OUTER => TRUE) d
  WHERE
    customer = 'woodpecker'
    AND CAST(created_at AS DATE) >= '2023-05-04'
    AND USER_EMAIL IN ('ivan.fernandez@contractors.scale.com', 'erik.verna@contractors.scale.com', 'andres.garcia@contractors.scale.com', 'astrid.hernandez@contractors.scale.com', 'vishal.kumar@scale.com')
'''
cs.execute(sql)
adf = cs.fetch_pandas_all()
#print columns of adf
print(adf.columns)

Index(['STATUS', 'CREATED_AT', 'USER_EMAIL', '_ID', 'PVID', 'ATTRIBUTE_NAME',
       'AUDIT_KEY', 'AUDITRESULT'],
      dtype='object')


In [31]:
#left join adf to mldf on PVID and attribute_name
mdf = pd.merge(mldf, adf, how='left', on=['PVID', 'ATTRIBUTE_NAME'])
#delete rows where AuditResult is null
mdf = mdf[mdf['AUDITRESULT'].notnull()]
#save mdf to csv file
mdf.to_csv('modelevalaudited.csv', index=False)